In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import pylab as plt

# Project

### Description

This is a learning project. The point of it is to run analysis on the HoI4 air combat algorithm. It will use python and MatPlotLib to make the calculations and display them as graphs to make analysis and comparison easier.

Learning note 1
Remember to run the code snippets with Shift-Enter. They also needs to be run in the correct order to update the scope when you make 

### Preparation

We need to set up the basic data that we will be running the algorithm over to see the results. We will be using objects to store and pass data. 

In [26]:
class Plane:
    def __init__(self, attack, defence, agility, speed, cost):
        self.attack = attack
        self.defence = defence
        self.agility = agility
        self.speed = speed
        self.cost = cost
    @property
    def attack(self):
        return self.attack
    @property
    def defence(self):
        return self.defence
    @property
    def agility(self):
        return self.agility
    @property
    def speed(self):
        return self.speed
    @property
    def cost(self):
        return self.cost
    @attack.setter
    def attack(self, val: int):
        self.weapons = val
    @defence.setter
    def defence(self, val: int):
        self.defence = val
    @agility.setter
    def agility(self, val: int):
        self.agility = val
    @speed.setter
    def speed(self, val: int):
        self.speed = val
    @cost.setter
    def cost 


This is the primary class we will be using. The properties are the values for the planes that we will be comparing.

In [63]:
class VariantPlane():
    def __init__(self, plane: Plane, weapons: int = 0):
        self.plane = plane
        self.weapons = weapons # +10% attack, - 1% speed, 10% Reliability, 5% agility
        self.range = 0 # +10% Range, -10% Reliability
        self.engine = 0 # +2% speed, 5% agility
        self.reliability = 0 # +10% Reliabilty
    @property
    def attack(self):
        return self.plane.attack + self.weapons
    @property
    def weapons(self):
        return self._weapons
    @weapons.setter
    def weapons(self, val: int):
        if not isinstance(val, int):
            raise TypeError("weapons should be an int")
        if val > 7:
            val = 7
        self._weapons = val


I will come back to figure out this later. The point is that I want to make variants to planes without having to type in everything. I will send in the plane I want a variant of and then overwrite the specific values.

I see that this might not be the best way to do this with this specific dataset. The upgrades are percentage bonus/penalties. That means that I want the original plane values and the upgrade points, and afterwards apply the bonuses.

I also want to be able to only set one of the possible variant values, and have the others initiate as Zero

In [68]:
def damage(attacker, defender):
    # damage = 0.01*number of attackers*air attack/air defence*(1+stats multiplier)*carrier factor*(1+agility disadvantage)
    multiplier = ((attacker.speed-defender.speed)/1500+(attacker.agility-defender.agility)/100)*1.3
    return multiplier

In [22]:
def test1():
    return (((500-430)/1500+(50-20)/100)*0.3)

print(test1())

def test2():
    return (((500-600)/1500+(50-60)/100)*0.3)

print(test2())

0.104
-0.05


In [70]:
f33 = Plane(9,8,45,330,22)
f36 = Plane(18,10,50,500,24)

print(damage(f33,f36))
print(damage(f36, f33))

-0.21233333333333335
0.21233333333333335


In [65]:
f33v1 = VariantPlane(plane=f33, weapons=50)

In [66]:
f33v1.__dict__

{'plane': <__main__.Plane at 0x1862ca134f0>,
 '_weapons': 7,
 'range': 0,
 'engine': 0,
 'reliability': 0}